In [1]:
# !pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 19.3 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=79f05f33828659a1f5fb436b706aff164375a338e41d4b62fc68c768f5d6349f
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [21]:
from pyspark.ml.classification import LogisticRegression
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.ml.feature import VectorAssembler
from sklearn.model_selection import train_test_split,cross_val_score,RepeatedStratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,f1_score,recall_score
from sklearn.model_selection import KFold 
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import VotingClassifier,BaggingClassifier,AdaBoostClassifier,RandomForestClassifier,ExtraTreesClassifier
from pyspark.ml.pipeline import PipelineModel
from pyspark.ml import Pipeline
%matplotlib inline

In [3]:
import pyspark as spark
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('ml-diab').getOrCreate()

In [5]:
url='/content/drive/MyDrive/diabetes_binary_5050split_health_indicators_BRFSS2015.csv'
# url='https://drive.google.com/uc?id=' + url.split('/')[-2]
df=spark.read.csv(url,inferSchema=True,header=True)

In [6]:
df.show()

+---------------+------+--------+---------+----+------+------+--------------------+------------+------+-------+-----------------+-------------+-----------+-------+--------+--------+--------+---+----+---------+------+
|Diabetes_binary|HighBP|HighChol|CholCheck| BMI|Smoker|Stroke|HeartDiseaseorAttack|PhysActivity|Fruits|Veggies|HvyAlcoholConsump|AnyHealthcare|NoDocbcCost|GenHlth|MentHlth|PhysHlth|DiffWalk|Sex| Age|Education|Income|
+---------------+------+--------+---------+----+------+------+--------------------+------------+------+-------+-----------------+-------------+-----------+-------+--------+--------+--------+---+----+---------+------+
|            0.0|   1.0|     0.0|      1.0|26.0|   0.0|   0.0|                 0.0|         1.0|   0.0|    1.0|              0.0|          1.0|        0.0|    3.0|     5.0|    30.0|     0.0|1.0| 4.0|      6.0|   8.0|
|            0.0|   1.0|     1.0|      1.0|26.0|   1.0|   1.0|                 0.0|         0.0|   1.0|    0.0|              0.0|   

In [7]:
df.describe().show()

+-------+------------------+-------------------+------------------+-------------------+------------------+------------------+-------------------+--------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+------------------+
|summary|   Diabetes_binary|             HighBP|          HighChol|          CholCheck|               BMI|            Smoker|             Stroke|HeartDiseaseorAttack|      PhysActivity|            Fruits|           Veggies|  HvyAlcoholConsump|      AnyHealthcare|        NoDocbcCost|           GenHlth|          MentHlth|          PhysHlth|           DiffWalk|               Sex|               Age|         Education|            Income|
+-------+------------------+-------------------+------------------+-------------------+------------------+------------------+-

In [8]:
df.dtypes

[('Diabetes_binary', 'double'),
 ('HighBP', 'double'),
 ('HighChol', 'double'),
 ('CholCheck', 'double'),
 ('BMI', 'double'),
 ('Smoker', 'double'),
 ('Stroke', 'double'),
 ('HeartDiseaseorAttack', 'double'),
 ('PhysActivity', 'double'),
 ('Fruits', 'double'),
 ('Veggies', 'double'),
 ('HvyAlcoholConsump', 'double'),
 ('AnyHealthcare', 'double'),
 ('NoDocbcCost', 'double'),
 ('GenHlth', 'double'),
 ('MentHlth', 'double'),
 ('PhysHlth', 'double'),
 ('DiffWalk', 'double'),
 ('Sex', 'double'),
 ('Age', 'double'),
 ('Education', 'double'),
 ('Income', 'double')]

In [9]:
len(df.columns[1:])

21

In [10]:
assembler=VectorAssembler(inputCols=df.columns[1:],outputCol='features')

In [11]:
final_df=assembler.transform(df)

In [12]:
final_df.select('features','Diabetes_binary').show()

+--------------------+---------------+
|            features|Diabetes_binary|
+--------------------+---------------+
|[1.0,0.0,1.0,26.0...|            0.0|
|[1.0,1.0,1.0,26.0...|            0.0|
|(21,[2,3,7,8,9,11...|            0.0|
|[1.0,1.0,1.0,28.0...|            0.0|
|(21,[2,3,4,7,8,9,...|            0.0|
|(21,[2,3,7,8,9,13...|            0.0|
|[0.0,1.0,1.0,26.0...|            0.0|
|(21,[2,3,4,8,9,11...|            0.0|
|(21,[2,3,7,8,9,11...|            0.0|
|(21,[2,3,4,8,9,11...|            0.0|
|[1.0,1.0,1.0,24.0...|            0.0|
|(21,[2,3,7,8,9,11...|            0.0|
|[1.0,1.0,1.0,27.0...|            0.0|
|[1.0,0.0,1.0,58.0...|            0.0|
|(21,[1,2,3,4,7,8,...|            0.0|
|(21,[2,3,4,7,8,11...|            0.0|
|(21,[2,3,7,9,11,1...|            0.0|
|(21,[2,3,4,7,8,9,...|            0.0|
|(21,[2,3,7,8,9,11...|            0.0|
|(21,[0,2,3,8,9,12...|            0.0|
+--------------------+---------------+
only showing top 20 rows



In [13]:
model_df = final_df.select(['features','Diabetes_binary'])
model_df = model_df.withColumnRenamed("Diabetes_binary","label")
model_df.printSchema()

root
 |-- features: vector (nullable = true)
 |-- label: double (nullable = true)



In [22]:
training_df,test_df = model_df.randomSplit([0.75,0.25])
log_reg=LogisticRegression()

In [23]:
model = log_reg.fit(training_df)

In [24]:
preds=model.transform(test_df)

In [25]:
preds.select('label','prediction').show(50)

+-----+----------+
|label|prediction|
+-----+----------+
|  0.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  0.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  0.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  0.0|       1.0|
|  1.0|       1.0|
|  0.0|       1.0|
|  0.0|       1.0|
|  1.0|       1.0|
|  0.0|       1.0|
|  1.0|       1.0|
|  0.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  0.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  0.0|       1.0|
|  0.0|       1.0|
|  0.0|       1.0|
|  0.0|       1.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  1.0|       0.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  0.0|       1.0|
|  1.0|       0.0|
|  0.0|       1.0|
|  0.0|       1.0|
|  0.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|     

In [26]:
preds.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|(21,[0,1,2,3,4,5,...|  0.0|[-0.5425622183828...|[0.36759174652488...|       1.0|
|(21,[0,1,2,3,4,5,...|  1.0|[-0.9060197203516...|[0.28781501560864...|       1.0|
|(21,[0,1,2,3,4,5,...|  1.0|[-0.7372412916359...|[0.32360769164237...|       1.0|
|(21,[0,1,2,3,4,5,...|  1.0|[-0.7180855778284...|[0.32781469096783...|       1.0|
|(21,[0,1,2,3,4,5,...|  0.0|[-0.7285772160135...|[0.32550702518896...|       1.0|
|(21,[0,1,2,3,4,6,...|  1.0|[-0.4663938462225...|[0.38547012621296...|       1.0|
|(21,[0,1,2,3,4,6,...|  1.0|[-0.0497303088299...|[0.48756998441329...|       1.0|
|(21,[0,1,2,3,4,6,...|  0.0|[-0.7086881412423...|[0.32988877779296...|       1.0|
|(21,[0,1,2,3,4,6,...|  1.0|[-0.8745087975035...|[0.29431698190966...|       1.0|
|(21,[0,1,2,3,4,

In [47]:
def get_f1(predictions):
  preds=predictions.select('prediction').rdd.flatMap(lambda x: x).collect()
  tru=predictions.select('label').rdd.flatMap(lambda x: x).collect()
  return f1_score(tru,preds)

In [49]:
print(f'F1 SCORE: {get_f1(preds)}')

F1 SCORE: 0.7535199510267684
